In [2]:
// Doric is still in snapshot only, adding sonatype snapshot repository
import coursierapi.MavenRepository

interp.repositories.update(
  interp.repositories() ::: List(MavenRepository.of("https://oss.sonatype.org/content/repositories/snapshots"))
)

import coursierapi.MavenRepository



# Doric
Spark developer
* forget "nulls" results in operations between invalid types
* forget the exception in "withColumn", target the real source
* forget running again and again to find all the erros


* keep all the spark optimizations
* keep your dataframe as dynamic as need


* and all this with a bear minimun change in your code
* only using it when you need it, full compatibility with your spark code

Let's check it out

In [3]:
import $ivy.`org.apache.spark::spark-sql:3.1.1`
import $ivy.`org.typelevel::cats-core:2.3.0`
import $ivy.`com.lihaoyi::sourcecode:0.2.6`
import $ivy.`org.hablapps::doric:0.0.0+79-f0c07327-SNAPSHOT`

Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-sql_2.12/3.1.1/spark-sql_2.12-3.1.1.pom
Downloaded https://repo1.maven.org/maven2/sh/almond/almond-spark_2.12/0.10.9/almond-spark_2.12-0.10.9.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-parent_2.12/3.1.1/spark-parent_2.12-3.1.1.pom
Downloaded https://repo1.maven.org/maven2/org/apache/apache/18/apache-18.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-core_2.12/3.1.1/spark-core_2.12-3.1.1.pom
Downloaded https://repo1.maven.org/maven2/org/apache/xbean/xbean-asm7-shaded/4.15/xbean-asm7-shaded-4.15.pom
Downloaded https://repo1.maven.org/maven2/org/apache/orc/orc-mapreduce/1.5.12/orc-mapreduce-1.5.12.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-catalyst_2.12/3.1.1/spark-catalyst_2.12-3.1.1.pom
Downloaded https://repo1.maven.org/maven2/org/apache/parquet/parquet-column/1.10.1/parquet-column-1.10.1.pom
Downloaded https://repo1.maven.org/maven2/sh/almond/ammoni

import $ivy.$                                  

import $ivy.$                               

import $ivy.$                              

import $ivy.$                                               

In [4]:
import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.functions.{col, lit}
import org.apache.spark.sql.{functions => f}
import habla.doric._
import habla.doric.{functions => doricf}

import org.apache.spark._

import org.apache.spark.sql._

import org.apache.spark.sql.functions.{col, lit}

import org.apache.spark.sql.{functions => f}

import habla.doric._

import habla.doric.{functions => doricf}

In [5]:
val spark = org.apache.spark.sql.SparkSession.builder().appName("test").master("local").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
21/06/03 18:45:59 INFO SparkContext: Running Spark version 3.1.1
21/06/03 18:46:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/06/03 18:46:00 INFO ResourceUtils: ==============================================================
21/06/03 18:46:00 INFO ResourceUtils: No custom resources configured for spark.driver.
21/06/03 18:46:00 INFO ResourceUtils: ==============================================================
21/06/03 18:46:00 INFO SparkContext: Submitted application: test
21/06/03 18:46:00 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
21/06/03 18:46:00 INFO ResourceProfile:

spark: SparkSession = org.apache.spark.sql.SparkSession@36265ca6

In [6]:
import spark.implicits._

import spark.implicits._

Doric accelerates the spark developer with three simple steps.
1. Column type validation
2. Typesafety of the operations between columns
3. Aggregation of multiple errors with improved location of the error place

## 1. Column Type validation

Spark works with columns. Until runtime the dataframe doesn't know the type, but the developer knows it, and can add a validation of the expected type. Imagine that we are expecting an dataframe with an id of type string, but in our flow we can't validate it.

In [7]:
spark.range(1,10).withColumn("x", f.concat(col("id"), lit("jander"))) // id is really a string, or spark is creating a implicit conversion?

res6: DataFrame = [id: bigint, x: string]

In [8]:
val df = spark.range(1,10).toDF

df.withColumn("x", doricf.concat(colString("id"), "jander".lit))

: 

As you can see, doric will treat as an error that the column type is invalid, this gives doric a lot of power, only changing the way that you get a column.

## 2. Typesafety of the operations with columns

Now that we have certinty that we will have a column of the expected type, or the program won't run, doric can prevent at compile time, errors like:

In [9]:
val dfEq = List((1, "1"), (1, " 1"), (1, " 1 ")).toDF("int", "str")

dfEq: DataFrame = [int: int, str: string]

In [10]:
dfEq.withColumn("eq", col("int") === col("str")).show

+---+---+----+
|int|str|  eq|
+---+---+----+
|  1|  1|true|
|  1|  1|true|
|  1| 1 |true|
+---+---+----+



Spark is telling me that an integer and a string are equal?? Eaven if the strings are different?? What kind of behaviour is this??
With doric, this code it woudn't compile.

In [10]:
dfEq.withColumn("eq", colInt("int") === colString("str")).show

cmd10.sc:1: type mismatch;
 found   : habla.doric.DoricColumn[String]
 required: habla.doric.DoricColumn[Int]
val res10 = dfEq.withColumn("eq", colInt("int") === colString("str")).show
                                                             ^Compilation Failed

: 

And if we were wrong, and expected "int" column as a string, in runtime we would get:

In [11]:
dfEq.withColumn("eq", colString("int") === colString("str")).show

: 

Learning spark is hard, we don't want to also learn all the possible places that can allow us invalid code as valid, and all the magic transformations, so let us do it in our way

In [12]:
dfEq.withColumn("eq", colInt("int").cast[String] === colString("str")).show

+---+---+-----+
|int|str|   eq|
+---+---+-----+
|  1|  1| true|
|  1|  1|false|
|  1| 1 |false|
+---+---+-----+



In [13]:
dfEq.withColumn("eq", colInt("int") === colString("str").unsafeCast[Int]).show
// its a warning cast to because you must know that it can put a null if it can be done

+---+---+----+
|int|str|  eq|
+---+---+----+
|  1|  1|true|
|  1|  1|true|
|  1| 1 |true|
+---+---+----+



This is a accelerator for the development, we have a located point of possible error in runtime, and if they pass it, it will run as expected. And with the knowledge of what function we can apply to what column.

## 3. Aggregation of multiple errors with improved location of the error place

Ok, we know what functions can be the reason to a runtime error, but, spark API is a fail fast API, and we have to run again and again to detect a single error, and when fixed, rerun again.

In [14]:
val dfadd = List((1,2),(3,4)).toDF("int1", "int2")

dfadd: DataFrame = [int1: int, int2: int]

In [15]:
dfadd.withColumn("add", col("Int_1") + col("Int_2")).show

: 

D'oh, i wrote wrong a column, let's fix it and run it again.

In [16]:
dfadd.withColumn("add", col("int1") + col("Int_2")).show

: 

next error... ok, let's fix it:

In [17]:
dfadd.withColumn("add", col("int1") + col("int2")).show

+----+----+---+
|int1|int2|add|
+----+----+---+
|   1|   2|  3|
|   3|   4|  7|
+----+----+---+



Doric can help us in this cases aggregating all the errors found in a dataframe transformation, in a single exception:

In [18]:
dfadd.withColumn("add", colInt("int_1") + colInt("int_2")).show

: 

Luckly my logic is very simple, and all my columns can fit in the withColumn call, but imagine that we have to split them, due to a separation of the logic and the execution:

In [19]:
val addColumns: Column = col("int_1") + col("int2")

dfadd.withColumn("add", addColumns).show

: 

I know its very simple to see in this logic, but the exception, marks the error in the `withColumn` method, but the real error is in other place, in the moment we asked for a unexisting column. Spark will give you a hint, but you must dive into your code to find the exact place.

With doric, we can simplify it, every method that can be the reason to an error is indexed and marked in the code.

In [20]:
val addColumns = colInt("int_1") + colInt("int2")

dfadd.withColumn("add", addColumns).show

: 

Let me copy the message of the error here again:

```
habla.doric.DoricMultiError: found 1 errors
Cannot resolve column name "int_1" among (int1, int2)
	located at . (cmd25.sc:1)
```

cmd25 is the name of the file, and the line number it contains the error is number 1, if Doric could, it would fix your code, but it has to leave something for the rest. And if you are using a IDE to develop, it will create a hiperlink to the line fo the error.

Lets try something "harder":

In [21]:
val col1 = colInt("int_1")
val col2 = colString("int2").unsafeCast[Int]
val addColumns = col1 + col2

dfadd.withColumn("add", addColumns).show

: 

See, each error explainded and located. Right on the target.

Ok, but will be asking for yourself, What is the dark side? Nothing, trully nothing, all your optimizations will work as pure Dataframe API, you still use your everydate Dataframe as normal if you want/need to.

## Alternatives to Doric

Doric is created to have a better and safer API for spark, but with the idea to be simple, and not a complete change for a spark developer.
Spark already has a typed API in scala with DataSet, but this API is not very frendly with all the optimizations.

We also have [Typelevel frameless](https://github.com/typelevel/frameless), that has a great API, that keeps the Dataset typed, and is fully compatible with the spark optimizations.

But bouth DataSet and Frameless TypedDataset share a common idea, to keep the whole schema allways. In both cases it has to be recreated with a case class. For example, if we expect our Dataset with 3 columns, 2 of them as string for the name and surname, and the third as an integer for the age, we would need to model it with the following:

```scala
case class User(name: String, surname: String, age: Int)

val ds: DataSet[User] = ???
```
If we want to enrich our row with the city, we have to use a new Structure, so we need to declare another case class

```scala
case class UserWithCity(name: String, surname: String, age: Int, city: String)

val ds2: DataSet[UserWithCity] = ds.???
```
In both dataset and typedDataset this is required, the thing that changes is the method to call.

This `case class` creation is acceptable if we don't have to transform the schema of the DataSet a lot, but if you are a spark develper, is not the normal case.

Other problem of this way to work, is that we cant do functions that transform a DataSet of an inteface, i mean, if we have different Datasets, that have different schemas but share a couple of columns that we whant to transform with the same function something like:

```scala
def parseTimestamp(df: DataFrame): DataFrame = 
   df.withColumn("timestampParsed", parse(df("timestamp")))
```

This can be seen as an inteface:

```scala
trait WithTimestamp {
    val timestamp: String
}

trait WithTimestampParsed {
    val timestampParsed: Long
}

def parseTimestamp[T <: WithTimestamp](df: Dataset[T]): Dataset[T with WithTimestampParsed] =
   df.withColumn("timestampParsed", parse(df("timestamp")))
```

I wish this code was as simple as i putted, but in the real world its very hard for the developer to do this.

And this is the reason to create Doric, its a balance between static and type safety for columns, but keeping the DataFrame as a dynamic data structure. Allowing the developer with previous knowledge of spark to jump in almost instantly, and for somebody thats getting into spark, skip the common erros for the beginer.

## Joins (doric join function)
Joins in spark are very coupled to the dataframes, for example for dataframes:

In [22]:
val leftdf = List((1,"hi"), (2, "bye")).toDF("id-left", "value-left")
val rightdf = List((1,"hi"), (2, "bye")).toDF("id-right", "value-right")

leftdf: DataFrame = [id-left: int, value-left: string]
rightdf: DataFrame = [id-right: int, value-right: string]

In [23]:
leftdf.join(rightdf, col("id-left") === col("id-right")).show

+-------+----------+--------+-----------+
|id-left|value-left|id-right|value-right|
+-------+----------+--------+-----------+
|      1|        hi|       1|         hi|
|      2|       bye|       2|        bye|
+-------+----------+--------+-----------+



This case we can do it because the columns to join have different names, but in case we have same name columns, we have to couple more the condition, using alias or the dataframes to extract the column reference.

In [24]:
val leftdf2 = List((1,"hi"), (2, "bye")).toDF("id", "value-left")
val rightdf2 = List((1,"hi"), (2, "bye")).toDF("id", "value-right")

leftdf2: DataFrame = [id: int, value-left: string]
rightdf2: DataFrame = [id: int, value-right: string]

In [25]:
leftdf2.alias("left")
.join(rightdf2.alias("right"), col("left.id") === col("right.id"))
.show

+---+----------+---+-----------+
| id|value-left| id|value-right|
+---+----------+---+-----------+
|  1|        hi|  1|         hi|
|  2|       bye|  2|        bye|
+---+----------+---+-----------+



In [26]:
leftdf2
.join(rightdf2, leftdf2("id") === rightdf2("id"))
.show

+---+----------+---+-----------+
| id|value-left| id|value-right|
+---+----------+---+-----------+
|  1|        hi|  1|         hi|
|  2|       bye|  2|        bye|
+---+----------+---+-----------+



Doric porpouse is to help you decouple your application and make it safer. This examples are very easy but imagine that id in each dataframe is of a different type, we will have the same problem that we had in previous examples. Also, as said, the condition is coupled to this two dataframes or alias.
Doric has a twist for this aproach with a doric join function, a condition that needs to match a left column element and a right column element, but not coupled to already defined dataframes.

In [27]:
val joinFunc = LeftDF.colInt("id") === RightDF.colInt("id")
leftdf2.join(rightdf2, joinFunc, "inner").show

+---+----------+---+-----------+
| id|value-left| id|value-right|
+---+----------+---+-----------+
|  1|        hi|  1|         hi|
|  2|       bye|  2|        bye|
+---+----------+---+-----------+



joinFunc: DoricJoinColumn = DoricJoinColumn(
  Kleisli(habla.doric.package$LeftDoricColumn$$Lambda$5377/276981435@bfa711f)
)

In cases that you need to preprocess your column, buth without creating one, is also very easy:

In [28]:
val leftdf2 = List(("1","hi"), ("2", "bye")).toDF("id", "value-left")
val rightdf2 = List((1,"hi"), (2, "bye")).toDF("id", "value-right")

leftdf2: DataFrame = [id: string, value-left: string]
rightdf2: DataFrame = [id: int, value-right: string]

In [29]:
val joinFunc2 = LeftDF.colString("id") === RightDF(colInt("id").cast[String])
leftdf2.join(rightdf2, joinFunc2, "inner").show

+---+----------+---+-----------+
| id|value-left| id|value-right|
+---+----------+---+-----------+
|  1|        hi|  1|         hi|
|  2|       bye|  2|        bye|
+---+----------+---+-----------+



joinFunc2: DoricJoinColumn = DoricJoinColumn(
  Kleisli(habla.doric.package$LeftDoricColumn$$Lambda$5377/276981435@43139be3)
)

And as you imagine, we stil have all the goodies that doric give us of type cheking and error location :D

In [30]:
val joinFunc2 = LeftDF.colInt("id") === RightDF.colInt("id-2")
leftdf2.join(rightdf2, joinFunc2, "inner").show

: 